In [4]:
import numpy as np
import pandas as pd
import datetime, re

# Getting data from already complied csv file
data = pd.read_csv('bitcoin.csv', encoding='utf-8', parse_dates=True, usecols=["Timestamp", "Close", "Weighted_Price"])

data['Weighted_Price'].fillna(value=0, inplace=True) # for filling NaN with 0 on days where there was no trade
data['Close'].fillna(method='ffill', inplace=True) # to forward propogate the previous values to NaN places

# Turtle Strategy was based on daily data with breakouts in a range of 20-55 days. Therefore we take range to be 55 days.
# As out data are in minutes, convert days into minutes
signal_lookback = 55 * 24 * 60 # days * hours * minutes

data['BuyOrSell'] = np.zeros(len(data)) # initializing the dataframe

data['RollingMax'] = data['Close'].shift(1).rolling(signal_lookback, min_periods=signal_lookback).max()
data['RollingMin'] = data['Close'].shift(1).rolling(signal_lookback, min_periods=signal_lookback).min()
data.loc[data['RollingMax'] < data['Close'], 'BuyOrSell'] = 1
data.loc[data['RollingMin'] > data['Close'], 'BuyOrSell'] = -1

print(data)
# ================ #
# DataFrame TABLE  #
# ================ #

                  Timestamp     Close  Weighted_Price  BuyOrSell  RollingMax  \
0       2017-01-01 00:00:00    967.34      967.372019        0.0         NaN   
1       2017-01-01 00:01:00    967.47      967.415156        0.0         NaN   
2       2017-01-01 00:02:00    966.65      967.333900        0.0         NaN   
3       2017-01-01 00:03:00    967.48      967.357170        0.0         NaN   
4       2017-01-01 00:04:00    966.62      967.479187        0.0         NaN   
5       2017-01-01 00:05:00    967.47      967.477987        0.0         NaN   
6       2017-01-01 00:06:00    966.81      967.315336        0.0         NaN   
7       2017-01-01 00:07:00    967.45      967.440768        0.0         NaN   
8       2017-01-01 00:08:00    967.49      967.370903        0.0         NaN   
9       2017-01-01 00:09:00    967.42      967.427125        0.0         NaN   
10      2017-01-01 00:10:00    967.65      967.551253        0.0         NaN   
11      2017-01-01 00:11:00    967.85   

In [ ]:
import plotly
import plotly.graph_objs as go
from plotly.offline import *
init_notebook_mode(connected=True)

# =================================================== #
# Ploting Closing Price, Rolling Max and Rolling Min  #
# =================================================== #

trace1 = go.Scatter(
    x=data['Timestamp'],
    y=data['Close'],
    mode = 'lines',
    name='Closing Price'
)
trace2 = go.Scatter(
    x=data['Timestamp'],
    y=data['RollingMax'],
    mode = 'lines',
    name='RollingMax'
)
trace3 = go.Scatter(
    x=data['Timestamp'],
    y=data['RollingMin'],
    mode = 'lines',
    name='RollingMin'
)
data = [trace1, trace2, trace3]
layout = go.Layout(
    title='Close, Rolling Max and Rolling Min - Bitcoin'
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='Bitcoin_close_rollingMax_Min.html')
# check above html files if the plot is not displayed

In [5]:
# ======================================================================================= #
# Ploting Closing Price and a Subplot showing where to Buy and to Sell (Turtle Strategy)  #
# ======================================================================================= #

trace1 = go.Scatter(
    x=data['Timestamp'],
    y=data['Close']
)
trace2 = go.Scatter(
    x=data['Timestamp'],
    y=data['BuyOrSell']
)
fig = plotly.tools.make_subplots(rows=4, cols=1, specs=[[{'rowspan': 3}], [None], [None], [{}]],
                          shared_xaxes=True, shared_yaxes=True,
                          subplot_titles=('BTC Price','Buy(+1) Or Sell(-1)'),
                          vertical_spacing=0.1)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 4, 1)

plotly.offline.iplot(fig, filename='bitcoin_turtle_analysis.html')
# check above html files if the plot is not displayed

This is the format of your plot grid:
[ (1,1) x1,y1 ]
       |       
       |       
[ (4,1) x1,y4 ]

